In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [2]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [3]:
import tensorflow as tf

In [4]:
tf.__version__

'2.0.0'

In [5]:
# To generate GIFs
!pip install imageio

You should consider upgrading via the 'pip install --upgrade pip' command.


In [6]:
# To use Differential Privacy
!pip install tensorflow_privacy

You should consider upgrading via the 'pip install --upgrade pip' command.


In [7]:
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers, models
import time

from IPython import display

from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy
from tensorflow_privacy.privacy.optimizers.dp_optimizer import DPGradientDescentGaussianOptimizer
from tensorflow_privacy.privacy.optimizers.dp_optimizer import DPAdamGaussianOptimizer

## Modified Optimizer

In [8]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from absl import logging

import tensorflow as tf

import collections

from tensorflow_privacy.privacy.analysis import privacy_ledger
from tensorflow_privacy.privacy.dp_query import gaussian_query
from tensorflow_privacy.privacy.dp_query import no_privacy_query


def make_optimizer_class(cls):
  """Constructs a DP optimizer class from an existing one."""
  parent_code = tf.compat.v1.train.Optimizer.compute_gradients.__code__
  child_code = cls.compute_gradients.__code__
  GATE_OP = tf.compat.v1.train.Optimizer.GATE_OP  # pylint: disable=invalid-name
  if child_code is not parent_code:
    logging.warning(
        'WARNING: Calling make_optimizer_class() on class %s that overrides '
        'method compute_gradients(). Check to ensure that '
        'make_optimizer_class() does not interfere with overridden version.',
        cls.__name__)

  class DPOptimizerClass(cls):
    """Differentially private subclass of given class cls."""

    _GlobalState = collections.namedtuple(
      '_GlobalState', ['l2_norm_clip', 'stddev'])
    
    def __init__(
        self,
        dp_sum_query,
        num_microbatches=None,
        unroll_microbatches=False,
        *args,  # pylint: disable=keyword-arg-before-vararg, g-doc-args
        **kwargs):
      """Initialize the DPOptimizerClass.

      Args:
        dp_sum_query: DPQuery object, specifying differential privacy
          mechanism to use.
        num_microbatches: How many microbatches into which the minibatch is
          split. If None, will default to the size of the minibatch, and
          per-example gradients will be computed.
        unroll_microbatches: If true, processes microbatches within a Python
          loop instead of a tf.while_loop. Can be used if using a tf.while_loop
          raises an exception.
      """
      super(DPOptimizerClass, self).__init__(*args, **kwargs)
      self._dp_sum_query = dp_sum_query
      self._num_microbatches = num_microbatches
      self._global_state = self._dp_sum_query.initial_global_state()
      # TODO(b/122613513): Set unroll_microbatches=True to avoid this bug.
      # Beware: When num_microbatches is large (>100), enabling this parameter
      # may cause an OOM error.
      self._unroll_microbatches = unroll_microbatches

    def compute_gradients(self,
                          loss,
                          var_list,
                          gate_gradients=GATE_OP,
                          aggregation_method=None,
                          colocate_gradients_with_ops=False,
                          grad_loss=None,
                          gradient_tape=None,
                          SET_noise=0,
                          SET_clip=10):

      self._dp_sum_query = gaussian_query.GaussianSumQuery(SET_clip, SET_clip*SET_noise)
      self._global_state = self._dp_sum_query.make_global_state(SET_clip, SET_clip*SET_noise)
      



      # TF is running in Eager mode, check we received a vanilla tape.
      if not gradient_tape:
        raise ValueError('When in Eager mode, a tape needs to be passed.')

      vector_loss = loss()
      if self._num_microbatches is None:
        self._num_microbatches = tf.shape(input=vector_loss)[0]
      sample_state = self._dp_sum_query.initial_sample_state(var_list)
      microbatches_losses = tf.reshape(vector_loss, [self._num_microbatches, -1])
      sample_params = (self._dp_sum_query.derive_sample_params(self._global_state))

      def process_microbatch(i, sample_state):
        """Process one microbatch (record) with privacy helper."""
        microbatch_loss = tf.reduce_mean(input_tensor=tf.gather(microbatches_losses, [i]))
        grads = gradient_tape.gradient(microbatch_loss, var_list)
        sample_state = self._dp_sum_query.accumulate_record(sample_params, sample_state, grads)
        return sample_state
    
      for idx in range(self._num_microbatches):
        sample_state = process_microbatch(idx, sample_state)

      if SET_noise > 0:
        grad_sums, self._global_state = (self._dp_sum_query.get_noised_result(sample_state, self._global_state))
      else:
        grad_sums = sample_state

      def normalize(v):
        return v / tf.cast(self._num_microbatches, tf.float32)

      final_grads = tf.nest.map_structure(normalize, grad_sums)
      grads_and_vars = final_grads#list(zip(final_grads, var_list))
    
      return grads_and_vars

  return DPOptimizerClass


def make_gaussian_optimizer_class(cls):
  """Constructs a DP optimizer with Gaussian averaging of updates."""

  class DPGaussianOptimizerClass(make_optimizer_class(cls)):
    """DP subclass of given class cls using Gaussian averaging."""

    def __init__(
        self,
        l2_norm_clip,
        noise_multiplier,
        num_microbatches=None,
        ledger=None,
        unroll_microbatches=False,
        *args,  # pylint: disable=keyword-arg-before-vararg
        **kwargs):
      dp_sum_query = gaussian_query.GaussianSumQuery(
          l2_norm_clip, l2_norm_clip * noise_multiplier)

      if ledger:
        dp_sum_query = privacy_ledger.QueryWithLedger(dp_sum_query,
                                                      ledger=ledger)

      super(DPGaussianOptimizerClass, self).__init__(
          dp_sum_query,
          num_microbatches,
          unroll_microbatches,
          *args,
          **kwargs)

    @property
    def ledger(self):
      return self._dp_sum_query.ledger

  return DPGaussianOptimizerClass


GradientDescentOptimizer = tf.compat.v1.train.GradientDescentOptimizer
DPGradientDescentGaussianOptimizer_NEW = make_gaussian_optimizer_class(GradientDescentOptimizer)

## START

In [9]:
base_dir = '/storage/TF2'

In [10]:
name_trial = '/touseexp'

result_dir = base_dir + '/GANPoster' + name_trial

In [11]:
# Define checkpoint dir and prefix
checkpoint_dir = result_dir + '/training_checkpoints'

def checkpoint_name(title):  
  checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt__" + str(title))
  return(checkpoint_prefix)

In [13]:
if not os.path.exists(result_dir):
  os.makedirs(result_dir)

if not os.path.exists(checkpoint_dir):
  os.makedirs(checkpoint_dir)

## C-GAN

In [14]:
def make_generator_model_FCC():
    # INPUT: label input
    in_label = layers.Input(shape=(COND_num_classes,))

    # INPUT: image generator input
    in_lat = layers.Input(shape=(Z_DIM,))

    # MERGE
    merge = layers.concatenate([in_lat, in_label], axis=1)

    ge1 = layers.Dense(128, use_bias=True)(merge)
    ge1 = layers.ReLU()(ge1)

    out_layer = layers.Dense(21, use_bias=True, activation="tanh")(ge1)

    model = models.Model([in_lat, in_label], out_layer)

    return model

def make_discriminator_model_FCC():
    # INPUT: Label
    in_label = layers.Input(shape=(COND_num_classes,))

    # INPUT: Image
    in_image = layers.Input(shape=(21,))

    # MERGE
    merge = layers.concatenate([in_image, in_label], axis=1)

    ge1 = layers.Dense(128, use_bias=True)(merge)
    ge1 = layers.ReLU()(ge1)

    out_layer = layers.Dense(1, use_bias=True)(ge1)

    model = models.Model([in_image, in_label], out_layer)

    return model

## SETUP

In [15]:
# SETUP
COND_num_classes = 3

Z_DIM = 100 # Does NOT affect EPSILON
NORM_CLIP = 1.1 # Does NOT affect EPSILON, but increases NOISE on gradients

DP_DELTA = 1e-5

NOISE_MULT = 1.15
EPOCHS = 50

BUFFER_SIZE = 3772

BATCH_SIZE = 32

N_DISC = 1

In [16]:
# Obtain DP_EPSILON
compute_dp_sgd_privacy.compute_dp_sgd_privacy(n = BUFFER_SIZE, 
                                              batch_size = BATCH_SIZE, 
                                              noise_multiplier = NOISE_MULT, 
                                              epochs = EPOCHS, 
                                              delta = DP_DELTA)

DP-SGD with sampling rate = 0.848% and noise_multiplier = 1.15 iterated over 5894 steps satisfies differential privacy with eps = 3.72 and delta = 1e-05.
The optimal RDP order is 7.0.


(3.718745314966683, 7.0)

In [17]:
# SD of noise
NOISE_MULT*NORM_CLIP

1.265

In [18]:
generator = make_generator_model_FCC()
discriminator = make_discriminator_model_FCC()

In [19]:
#generator.summary()

In [20]:
#discriminator.summary()

### OPTIMIZERS

In [21]:
import tensorflow.keras.backend as K

In [22]:
###################
## OPTIMIZERS
generator_optimizer = tf.keras.optimizers.Adam()

lr_disc = tf.compat.v1.train.polynomial_decay(learning_rate=0.150,
                                              global_step=tf.compat.v1.train.get_or_create_global_step(),
                                              decay_steps=10000,
                                              end_learning_rate=0.052,
                                              power=1)

discriminator_optimizer = DPGradientDescentGaussianOptimizer_NEW(
   learning_rate = lr_disc,
   l2_norm_clip = NORM_CLIP,
   noise_multiplier = NOISE_MULT,
   num_microbatches = BATCH_SIZE)

# DP
cross_entropy_DISC = tf.keras.losses.BinaryCrossentropy(from_logits=True, reduction=tf.losses.Reduction.NONE)

# Not-DP
cross_entropy_GEN = tf.keras.losses.BinaryCrossentropy(from_logits=True)

###################
## LOSS AND UPDATES

# Notice the use of `tf.function`: This annotation causes the function to be "compiled".
@tf.function
def train_step_DISC_BOTH(images, labels, noise):    
    with tf.GradientTape(persistent=True) as disc_tape_real:
      # This dummy call is needed to obtain the var list.
      dummy = discriminator([images, labels], training=True)
      var_list = discriminator.trainable_variables

      # In Eager mode, the optimizer takes a function that returns the loss.
      def loss_fn_real():
        real_output = discriminator([images, labels], training=True)
        disc_real_loss = cross_entropy_DISC(tf.ones_like(real_output), real_output)
        return disc_real_loss
      
      grads_and_vars_real = discriminator_optimizer.compute_gradients(loss_fn_real, var_list, 
                                                                      gradient_tape=disc_tape_real, 
                                                                      SET_noise=NOISE_MULT,
                                                                      SET_clip=NORM_CLIP)
      
      # In Eager mode, the optimizer takes a function that returns the loss.
      def loss_fn_fake():
        generated_images = generator([noise, labels], training=True)
        fake_output = discriminator([generated_images, labels], training=True)
        disc_fake_loss = cross_entropy_DISC(tf.zeros_like(fake_output), fake_output)
        return disc_fake_loss
      
      grads_and_vars_fake = discriminator_optimizer.compute_gradients(loss_fn_fake, var_list, 
                                                                      gradient_tape=disc_tape_real,
                                                                      SET_noise=0,
                                                                      SET_clip=NORM_CLIP)
    
    norm_r = 0 #tf.linalg.global_norm(grads_and_vars_real)
    norm_f = 0 #tf.linalg.global_norm(grads_and_vars_fake)
        
    s_grads_and_vars = [(grads_and_vars_real[idx] + grads_and_vars_fake[idx]) for idx in 
                        range(len(grads_and_vars_real))]
    sanitized_grads_and_vars = list(zip(s_grads_and_vars, var_list))
    
    discriminator_optimizer.apply_gradients(sanitized_grads_and_vars)
    
    return(norm_r, norm_f)


################################################################################################
################################################################################################

# Notice the use of `tf.function`: This annotation causes the function to be "compiled".
@tf.function
def train_step_GEN(labels, noise):
    with tf.GradientTape() as gen_tape:
      generated_images = generator([noise, labels], training=True)
      fake_output = discriminator([generated_images, labels], training=True)
      # if the generator is performing well, the discriminator will classify the fake images as real (or 1)
      gen_loss = cross_entropy_GEN(tf.ones_like(fake_output), fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))

    return(gen_loss)

## TRAIN

In [23]:
# CREATE CHECKPOINT STRUCTURE
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [24]:
def train_main_loop(dataset, title):
  for epoch in range(EPOCHS):
    start = time.time()

    i_gen = 0
    for image_batch, label_batch in dataset:      
      noise = tf.random.normal([BATCH_SIZE, Z_DIM])
      #print(str(i_gen) + " ------------------------------------------------------ ")
      d_r, d_f = train_step_DISC_BOTH(image_batch, label_batch, noise)
      #print("------ NORM REAL: " + str(d_r))
      #print("------ NORM FAKE: " + str(d_f))

      #if (i_gen + 1) % N_DISC == 0:
      g_f = train_step_GEN(label_batch, noise)
      
      i_gen = i_gen + 1      

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

    # Save the model
    checkpoint.save(file_prefix = checkpoint_name(title + "__epoch=" + str(epoch) + "__"))

## Dataset

In [25]:
import pandas as pd

In [26]:
train = pd.read_csv("/storage/TF2/ann-train.data", sep=" ",  header=None)
train.drop(train.columns[[22, 23]], axis=1, inplace=True)
trainXpre = train.loc[:,range(21)].values
trainYpre = train.loc[:,21]

In [27]:
trainXpre

array([[0.73 , 0.   , 1.   , ..., 0.12 , 0.082, 0.146],
       [0.24 , 0.   , 0.   , ..., 0.143, 0.133, 0.108],
       [0.47 , 0.   , 0.   , ..., 0.102, 0.131, 0.078],
       ...,
       [0.88 , 0.   , 0.   , ..., 0.123, 0.099, 0.124],
       [0.64 , 1.   , 0.   , ..., 0.106, 0.088, 0.121],
       [0.46 , 0.   , 0.   , ..., 0.093, 0.091, 0.102]])

In [28]:
from sklearn.preprocessing import OneHotEncoder

trainYpre = trainYpre.values
onehot_encoder = OneHotEncoder(sparse=False)
trainYpre = trainYpre.reshape(len(trainYpre), 1)
trainY = onehot_encoder.fit_transform(trainYpre)
print(trainY)

[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 ...
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [29]:
from sklearn import preprocessing

trainX = preprocessing.maxabs_scale(trainXpre)
trainX

array([[0.77659574, 0.        , 1.        , ..., 0.27906977, 0.35344828,
        0.23856209],
       [0.25531915, 0.        , 0.        , ..., 0.33255814, 0.57327586,
        0.17647059],
       [0.5       , 0.        , 0.        , ..., 0.2372093 , 0.56465517,
        0.12745098],
       ...,
       [0.93617021, 0.        , 0.        , ..., 0.28604651, 0.42672414,
        0.20261438],
       [0.68085106, 1.        , 0.        , ..., 0.24651163, 0.37931034,
        0.19771242],
       [0.4893617 , 0.        , 0.        , ..., 0.21627907, 0.39224138,
        0.16666667]])

In [30]:
print(trainX.shape)
print(trainY.shape)

(3772, 21)
(3772, 3)


# Train

In [32]:
tf.random.set_seed(1)
train_dataset = tf.data.Dataset.from_tensor_slices((trainX, trainY)).shuffle(BUFFER_SIZE, reshuffle_each_iteration=True).batch(BATCH_SIZE, drop_remainder=True)

In [33]:
training_title = 'TD_first'

In [34]:
train_main_loop(train_dataset, training_title)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Time for epoch 1

## Classification

In [35]:
checkpoint_dir

'/storage/TF2/GANPoster/touseexp/training_checkpoints'

In [36]:
checkpoint.restore('/storage/TF2/GANPoster/touseexp/training_checkpoints/ckpt__TD_first__epoch=49__-50')

In [37]:
tf.random.set_seed(100)
N_GEN = 3772

noise_GEN = tf.random.normal([N_GEN, Z_DIM])
labels_GEN = tf.Variable(np.array([1,0,0]*93 + 
                                   [0,1,0]*191 +
                                   [0,0,1]*3488, 
                                   dtype='float32').reshape((N_GEN,COND_num_classes)))

In [38]:
images_GEN = generator([noise_GEN, labels_GEN], training=False)
images_tt = layers.Flatten()(images_GEN)

In [39]:
labels_tt = tf.Variable(np.array([0]*93 + 
                                   [1]*191 +
                                   [2]*3488, 
                                   dtype='float32').reshape((N_GEN,1)))

In [40]:
print(labels_tt.shape)
images_tt.shape

(3772, 1)


TensorShape([3772, 21])

In [42]:
Y_train = labels_tt[:images_tt.shape[0]]
X_train = images_tt

### Cross Validation

In [43]:
!pip install sklearn

You should consider upgrading via the 'pip install --upgrade pip' command.


In [44]:
import numpy as np
import pandas as pd

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

import time

In [45]:
start = time.time()
parameters = {'solver': ['lbfgs', 'adam'], 
              'max_iter': [300], 
              'alpha': 10.0 ** -np.arange(1, 10), 
              'hidden_layer_sizes': np.arange(15, 20), 
              'random_state':[1]}
clf = GridSearchCV(MLPClassifier(), parameters, n_jobs=-1)

clf.fit(trainX, trainY)
print(clf.score(trainX, trainY))
print(clf.best_params_)
print(time.time()-start)

0.9915164369034994
{'alpha': 1e-08, 'hidden_layer_sizes': 16, 'max_iter': 300, 'random_state': 1, 'solver': 'lbfgs'}
112.29233694076538


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


## TEST RESULTS

In [46]:
test = pd.read_csv("/storage/TF2/ann-test.data", sep=" ",  header=None)
test.drop(test.columns[[22, 23]], axis=1, inplace=True)
testXpre = test.loc[:,range(21)]
testX = preprocessing.maxabs_scale(testXpre)

testYpre = test.loc[:,21]

In [47]:
from sklearn.preprocessing import OneHotEncoder

testYpre = testYpre.values
onehot_encoder = OneHotEncoder(sparse=False)
testYpre = testYpre.reshape(len(testYpre), 1)
testY = onehot_encoder.fit_transform(testYpre)
print(testY)

[[0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [48]:
# Accuracy
clf.score(testX, testY)

0.956242707117853

In [49]:
from sklearn import metrics
from sklearn.preprocessing import LabelBinarizer

def multiclass_roc_auc_score(y_test, y_pred, average="micro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)
    return metrics.roc_auc_score(y_test, y_pred, average=average)

In [50]:
# AuROC
multiclass_roc_auc_score(testY, clf.predict(testX))

0.9746207701283548